In [ ]:
# plotter for the Wiener SVD unfolding output ROOT file
# background subtract before running through WienerSVD
# compute the cross section after computing the unfolded spectrum

In [ ]:
import sys
sys.path.insert(0, 'backend_functions')

import importlib

import uproot
import matplotlib.pylab as pylab
import numpy as np
import math

import awkward
import matplotlib.pyplot as plt
import pandas as pd

import ROOT
from ROOT import TH1F, TDirectory

import os

import top 
from top import *

In [ ]:
ISRUN3 = None

if ISRUN3: 
    print("Using RHC values !! ")

f = uproot.open('/uboone/data/users/kmiller/uBNuMI_CCNp/unfolding/output_shr_energy_cali_Combined_July15.root')

In [ ]:
rho_argon = 1.3836 # g/cm^3
fv = 236*212*1016
n_a = 6.022E23
n_nucleons = 40
m_mol = 39.95 #g/mol

n_target = (rho_argon * fv * n_a * n_nucleons) / m_mol

if ISRUN3==None: 
    cv_flux = (2E20*1.1864531e-11 + 5.014E20*8.6283762e-12)
    
else: 
    cv_flux = parameters(ISRUN3)['integrated_flux_per_pot']*parameters(ISRUN3)['beamon_pot']
    

# input spectra (true & measured)

In [ ]:
h_meas = f["hmeas"]
h_true = f['htrue_signal']

In [ ]:
bins = [round(var, 2) for var in h_meas.edges]
x_ticks = [0.02, 0.22, 0.42, 0.62, 0.82 , 1.22, 2.5]

bincenters = 0.5*(np.array(x_ticks)[1:]+np.array(x_ticks)[:-1])

b_meas = bins
b_true = bins

In [ ]:
binwidth = []

for x in range(len(bincenters)): 
    binwidth.append(round(abs(x_ticks[x+1]-x_ticks[x])/2, 2))
    
binwidth

In [ ]:
meas = h_meas.values
true = h_true.values

In [ ]:
fig = plt.figure(figsize=(10, 7))

plt.hist(bincenters, bins, color='blue', label='Reco (Selected)', alpha=0.4, weights=meas)
plt.hist(bincenters, bins, color='orange', label='True (Generated)', alpha=0.4, weights=true)

plt.legend(fontsize=13)
plt.title('Signal Input Distributions', fontsize=16)

#plt.ylim(0, 5E-40)
plt.xlim(0, 2.5)

plt.xlabel("Shower Energy [GeV]", fontsize=15)
    
plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

#plt.savefig(plots_path+"_InputSignalDistributions_ElectronEnergy.pdf", transparent=True, bbox_inches='tight') 

plt.grid()
plt.show()


## input covariance matrix 

In [ ]:
h_cov = f['hcov_tot']

In [ ]:
b_cov = list(h_cov.edges[0])
cov = h_cov.values

In [ ]:
fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(b_cov, b_cov, cov, cmap='OrRd', edgecolors='k' )

cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

#cbar.set_label("$\\tilde{\\sigma}^{2}$ [cm$^{4}$ / nucleon$^{2}$]", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=12)
plt.yticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=12)

plt.xlim(0.02, 2.5)
plt.ylim(.02, 2.5)

plt.xlabel("Reco Shower Energy [GeV]", fontsize=15)
plt.ylabel("Reco Shower Energy [GeV]", fontsize=15)

plt.title('Absolute Covariance', fontsize=15)

plt.show()

## input response matrix 

In [ ]:
h_res = f['hR']

In [ ]:
b_res = list(h_res.edges[0])
res = h_res.values

In [ ]:
fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(b_res, b_res, res, cmap='OrRd', edgecolors='k')

cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
#cbar.set_label("$\\nu$ / 2 $\\times$ 10$^{20}$ POT (normalized)", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=12)
plt.yticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=12)

plt.xlim(0.02, 2.5)
plt.ylim(.02, 2.5)


plt.xlabel("True Electron Energy [GeV]", fontsize=15)
plt.ylabel("Reco Electron Energy [GeV]", fontsize=15)

plt.title('Input Response Matrix', fontsize=15)

#plt.savefig(plots_path+"_InputResponseMatrix_ElectronEnergy.pdf", transparent=True, bbox_inches='tight') 

plt.show()

# unfolded result

In [ ]:
h_unf = f['unf']
unf = list(h_unf.values)

In [ ]:

fig = plt.figure(figsize=(8, 5))

plt.step(b_meas+[b_meas[-1]], [0]+list(meas)+[0], alpha=0.4, color='blue')
plt.fill_between(b_meas+[b_meas[-1]],[0]+list(meas)+[0], step="pre", alpha=0.4, label='reco (input)')


plt.step(b_meas+[b_meas[-1]], [0]+list(true)+[0], color='orange')
plt.fill_between(b_meas+[b_meas[-1]],[0]+list(true)+[0], step="pre", alpha=0.4, label='true (input)')


plt.step(b_meas+[b_meas[-1]], [0]+unf+[0], color='black', linewidth=2, linestyle='--', label='unfolded (output)')

plt.legend(fontsize=13)

plt.xlabel("Electron Energy [GeV]", fontsize=15)

plt.xlim(0, 2.5)
plt.ylim(0, 400)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

plt.legend(fontsize=13, loc='upper right')
plt.title('Unfolded Evt Rate', fontsize=16)
plt.show()

In [ ]:
# covariance of unfolded result 

h_unfcov = f['unfcov']


In [ ]:
b_unfcov = list(h_unfcov.edges[0])
unfcov = h_unfcov.values

In [ ]:
fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(b_unfcov, b_unfcov, unfcov, cmap='OrRd', edgecolors='k')#, vmin=0, vmax=1E-81)

cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlabel("Unfolded Electron Energy [GeV]", fontsize=15)
plt.ylabel("Unfolded Electron Energy [GeV]", fontsize=15)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=12)
plt.yticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=12)

plt.xlim(0.02, 2.5)
plt.ylim(.02, 2.5)

plt.title('Covariance of Unfolded Result', fontsize=15)

#plt.savefig(plots_path+"_UnfoldedCovariance_ElectronEnergy.pdf", transparent=True, bbox_inches='tight') 


plt.show()

In [ ]:
# fractional uncertainty 

h_frac = f['fracError']
b_frac = [round(var, 2) for var in h_frac.edges]

frac = list(h_frac.values)

In [ ]:
fig = plt.figure(figsize=(8, 5))

# calculate the width of each bin 
#x_err = [ (b_frac[i+1]-b_frac[i])/2 for i in range(len(b_frac)-1) ]
#plt.errorbar(b_frac_centers, frac, linestyle="", xerr=x_err, color="royalblue", label='fractional uncertainty')

plt.step(bins+[bins[-1]], [0]+frac+[0], color="royalblue")

plt.xlabel("Unfolded Electron Energy [GeV]", fontsize=15)

#plt.ylim(0, 1)

plt.title("Fractional Uncertainty of Unfolded Result", fontsize=15)
plt.show()

In [ ]:
# fractional difference between model & unfolded result

In [ ]:
h_diff = f['diff']

In [ ]:
b_diff = list(h_diff.edges)
diff = list(h_diff.values)

In [ ]:
fig = plt.figure(figsize=(8, 5))

plt.errorbar(bincenters, diff, color="royalblue", fmt='.', xerr=binwidth, markersize=1)

plt.xlim(0, 3)
plt.ylim(-0.2, 0)

plt.xlabel("Electron Energy [GeV]", fontsize=15)

plt.title("Fractional Difference between True & Unfolded Spectra", fontsize=15)
plt.show()

### Regularization Matrix

In [ ]:
# regularization matrix A_c 

h_Ac = f['smear']

In [ ]:
b_Ac = list(h_Ac.edges[0])
Ac = h_Ac.values

In [ ]:
fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(b_Ac, b_Ac, Ac, cmap='OrRd', edgecolors='k')

cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=12)
plt.yticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=12)

plt.xlim(0.02, 2.5)
plt.ylim(.02, 2.5)


#plt.xlabel("cos $\\theta_{ep}$", fontsize=15)
#plt.ylabel("cos $\\theta_{ep}$", fontsize=15)

plt.title('Additional Smearing Matrix ($A_{c}$)', fontsize=15)
#plt.savefig(plots_path+"Regularization.pdf", transparent=True, bbox_inches='tight') 

plt.show()

## Smear the true distribution by Ac

In [ ]:
smeared = []
s = 0

for i in range(len(b_Ac)-1):
    # compute the smeared bin value 
    for j in range(len(b_Ac)-1): 
        s = s + Ac[i][j]*true[j]

    smeared.append(s)
    s = 0
    

In [ ]:
fig = plt.figure(figsize=(8, 5))
   
plt.step(bins+[bins[-1]], [0]+unf+[0], color='black', 
             linewidth=2, linestyle='--', label='unfolded')
    
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)


plt.title('Unfolded Result', fontsize=16)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 5))

 ## chi2 calculation ## 
chi2 = 0 
# for i,j in zip(n[-1], n_data): 
    # if i>0: 
        #chi2 = chi2 + ((i-j)*(i-j))/i

    
#for i,j in zip(list(np.array(smeared)/np.array(binwidth)), np.array(unf)/np.array(binwidth)):
    # print(i, j)
    # print(((i-j)*(i-j))/i)
#    chi2 = chi2 + ((i-j)*(i-j))/i
    
    
plt.fill_between(bins+[bins[-1]],[0]+smeared+[0], step="pre", alpha=0.7, label='true (smeared)', color='mediumseagreen')
plt.step(bins+[bins[-1]], [0]+smeared+[0], color='mediumseagreen')
    
plt.step(bins+[bins[-1]], [0]+unf+[0], color='black', linewidth=2, linestyle='--', label='unfolded')
    
plt.ylim(0, 300)
plt.xlim(0, 2.5)

plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks( fontsize=13)


plt.xlabel("Electron Energy [GeV]", fontsize=15)

plt.legend(fontsize=13)
plt.title('FHC+RHC Unfolded Event Rate', fontsize=16)
#plt.savefig(plots_path+"UnfoldedDifferential_VisibleEnergy.pdf", transparent=True, bbox_inches='tight') 


plt.show()

## Convert to xsec units

In [ ]:
# divide by bin width, flux, and n_target 
from matplotlib import gridspec

In [ ]:
fig = plt.figure(figsize=(8, 7))

gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])

ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])
    
ax1.tick_params(axis = 'both', which = 'major', labelsize = 14)
ax2.tick_params(axis = 'both', which = 'major', labelsize = 14)

ax1.yaxis.grid(linestyle="--", color='black', alpha=0.2)
ax1.xaxis.grid(linestyle="--", color='black', alpha=0.2)


h_smeared = ax1.hist(bincenters, bins, color='mediumseagreen', label='true (smeared)', range=[bins[0],bins[-1]], 
             weights=1E39*(np.array(smeared)/(n_target*cv_flux*np.array(binwidth))), alpha=0.7)

result_xsec = list(1E39*np.array(unf)/(n_target*cv_flux*np.array(binwidth)))

#ax1.step(bins+[bins[-1]], [0]+result_xsec+[0], color='black', 
#             linewidth=2, linestyle='--', label='unfolded')

ax1.errorbar(bincenters, result_xsec, 
             xerr=binwidth, yerr=[x*y for x,y in zip(frac, result_xsec)], fmt='o', 
             color='black', ecolor='black', markersize=3, label='unfolded') 

# ratio plot  
ax2.errorbar(bincenters, result_xsec/h_smeared[0], #yerr=get_ratio_err(n_data, n[-1]), 
             xerr=binwidth, color="black", fmt='o')
ax2.set_ylim(0.99, 1.01)

ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)
    
    
ax1.set_ylabel("d$\\sigma$/dE$_{e}$\n[$\\times 10^{-39}$ cm$^{2}$ / GeV / nucleon]", fontsize=14)
ax1.set_xlim(0, 2.5)
ax1.set_xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22])

ax2.set_xlim(0, 2.5)
ax2.set_xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22])

ax2.set_xlabel("Shower Energy [GeV]", fontsize=14)
ax2.set_ylabel("Unfolded / True", fontsize=14)

ax1.legend(fontsize=13)

if ISRUN3==None: 
    ax1.set_title('FHC+RHC Cross Section', fontsize=16)
    
elif ISRUN3==False: 
    ax1.set_title('FHC Cross Section', fontsize=16)
    
elif ISRUN3==True: 
    ax1.set_title('RHC Cross Section', fontsize=16)


plt.show()

In [ ]:
results_dict = {
    'cv_evtrate' : smeared, 
    'cv_xsec' : result_xsec, 
    'covariance' : unfcov.tolist(), 
    'uncertainty' : [x*y for x,y in zip(frac, result_xsec)], 
    'fractional_uncertainty' : frac
}

## save to file

In [ ]:
ISRUN3

In [ ]:
# save to / update json file 

import io
import json
import os
from datetime import date
 
if ISRUN3==None: 
    file = 'CombinedClosureTest_ElectronEnergy_'+date.today().strftime("%m%d%y")+'.json'

elif ISRUN3==False: 
    file = 'FHCClosureTest_ElectronEnergy_'+date.today().strftime("%m%d%y")+'.json'
    
elif ISRUN3==True: 
    file = 'RHCClosureTest_ElectronEnergy_'+date.today().strftime("%m%d%y")+'.json'
    
      
print(file)
    
path = '/uboone/app/users/kmiller/searchingfornues_v33/uBNuMI_CC1eNp/unfolding/'


In [ ]:
if os.path.exists(path+file): 
    print(file, " exists and is readable, need to update file name to save ! ")
        
else: 
    with open('unfolding/'+file, 'w') as f:
            json.dump(results_dict, f)
    
    

## Comparisons

In [ ]:
with open('unfolding/FHCClosureTest_ElectronEnergy_071522.json') as f_fhc:
    fhc_dict = json.load(f_fhc)
    
with open('unfolding/RHCClosureTest_ElectronEnergy_071522.json') as f_rhc:
    rhc_dict = json.load(f_rhc)

with open('unfolding/CombinedClosureTest_ElectronEnergy_071522.json') as f:
    comb_dict = json.load(f)

In [ ]:
fig = plt.figure(figsize=(8, 5))

plt.errorbar(bincenters, fhc_dict['cv_xsec'], 
             xerr=binwidth, yerr=fhc_dict['uncertainty'], fmt='o', markersize=3, label='FHC') 

plt.errorbar(bincenters, rhc_dict['cv_xsec'], 
             xerr=binwidth, yerr=rhc_dict['uncertainty'], fmt='o', markersize=3, label='RHC', color='orange') 

plt.errorbar(bincenters, comb_dict['cv_xsec'], 
             xerr=binwidth, yerr=comb_dict['uncertainty'], fmt='o', markersize=3, label='FHC+RHC', color='black') 

plt.ylabel("d$\\sigma$/dE$_{e}$\n[$\\times 10^{-39}$ cm$^{2}$ / GeV / nucleon]", fontsize=14)
plt.xlim(0, 2.5)
plt.xticks([0.02, 0.22, 0.42, 0.62, 0.82 , 1.22], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Shower Energy [GeV]", fontsize=14)

plt.grid(linestyle="--", color='black', alpha=0.2)

plt.title("Closure Tests", fontsize=16)

plt.legend(fontsize=14)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 5))  

plt.hist(bincenters, bins, histtype='step', label="FHC", weights=fhc_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', label="RHC", weights=rhc_dict['fractional_uncertainty'], color='orange', 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', label="FHC+RHC", weights=comb_dict['fractional_uncertainty'], color='black', 
        linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Unfolded Shower Energy [GeV]", fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], 2.5)
plt.ylim(0, 0.5)

plt.legend(fontsize=14)
plt.title("Fractional Uncertainty", fontsize=16)

plt.show()